https://qiita.com/akita0724/items/694cc4e142d8d1d7fdaa

| 試験区分                         | コード |
|-----------------------------------|------|
| 応用情報技術者試験                 | ap    |
| ITストラテジスト試験               | st    |
| システムアーキテクト試験             | sa    |
| プロジェクトマネージャ試験           | pw    |
| ネットワークスペシャリスト試験       | nw    |
| データベーススペシャリスト試験         | db    |
| エンベデッドシステムスペシャリスト試験 | es    |
| ITサービスマネージャ試験           | sm    |
| システム監査技術者試験             | au    |
| 情報処理安全確保支援士試験           | sc    |

In [5]:
import requests  # HTTPリクエストを送信するためのライブラリ
from os import makedirs, path  # ディレクトリ操作やファイルパス操作を行うためのライブラリ
from time import sleep  # 一定時間処理を停止させるためのライブラリ
from urllib.request import urlretrieve  # URLからファイルをダウンロードするためのライブラリ

# 試験区分とコードの対応表
# ユーザーが選択した試験区分に対応する日本語名を取得するために使用
exam_codes = {
    "ap": "応用情報技術者試験",
    "st": "ITストラテジスト試験",
    "sa": "システムアーキテクト試験",
    "pw": "プロジェクトマネージャ試験",
    "nw": "ネットワークスペシャリスト試験",
    "db": "データベーススペシャリスト試験",
    "es": "エンベデッドシステムスペシャリスト試験",
    "sm": "ITサービスマネージャ試験",
    "au": "システム監査技術者試験",
    "sc": "情報処理安全確保支援士試験",
}

# 過去問のURLを生成するための情報
base_url = "https://www.ipa.go.jp/shiken/mondai-kaiotu/"  # 過去問のURLの共通部分
exam_data = [
    # 過去問のIDとファイル名プレフィックスのリスト
    # 各試験年度や季節ごとに必要な情報を格納
    {"id": "m42obm000000afqx-att", "filename_prefix": "2024r06h_"},
    {"id": "ps6vr70000010d6y-att", "filename_prefix": "2023r05a_"},
    {"id": "ps6vr70000010d6y-att", "filename_prefix": "2023r05h_"},
    {"id": "gmcbt80000008smf-att", "filename_prefix": "2022r04a_"},
    {"id": "gmcbt80000009sgk-att", "filename_prefix": "2022r04h_"},
    {"id": "gmcbt8000000apad-att", "filename_prefix": "2021r03a_"},
    {"id": "gmcbt8000000d5ru-att", "filename_prefix": "2021r03h_"},
    {"id": "gmcbt8000000d05l-att", "filename_prefix": "2020r02o_"},
    {"id": "gmcbt8000000dict-att", "filename_prefix": "2019r01a_"},
    {"id": "gmcbt8000000ddiw-att", "filename_prefix": "2019h31h_"},
    {"id": "gmcbt8000000f01f-att", "filename_prefix": "2018h30a_"},
    {"id": "gmcbt8000000fabr-att", "filename_prefix": "2018h30h_"},
    {"id": "gmcbt8000000fqpm-att", "filename_prefix": "2017h29a_"},
    {"id": "gmcbt8000000fzx1-att", "filename_prefix": "2017h29h_"},
    {"id": "gmcbt8000000g6fw-att", "filename_prefix": "2016h28a_"},
    {"id": "gmcbt8000000gn5o-att", "filename_prefix": "2016h28h_"},
    {"id": "gmcbt8000000gxj0-att", "filename_prefix": "2015h27a_"},
    {"id": "ug65p90000000f52-att", "filename_prefix": "2015h27h_"},
    {"id": "ug65p90000000ye5-att", "filename_prefix": "2014h26a_"},
    {"id": "ug65p90000001dzu-att", "filename_prefix": "2014h26h_"},
    {"id": "ug65p900000027za-att", "filename_prefix": "2013h25a_"},
    {"id": "ug65p90000002e6g-att", "filename_prefix": "2013h25h_"},
    {"id": "ug65p90000002h5m-att", "filename_prefix": "2012h24a_"},
    {"id": "ug65p900000038er-att", "filename_prefix": "2012h24h_"},
    {"id": "ug65p90000003ojp-att", "filename_prefix": "2011h23a_"},
    {"id": "ug65p90000003ya2-att", "filename_prefix": "2011h23tokubetsu_"},
    {"id": "ug65p90000004d6f-att", "filename_prefix": "2010h22a_"},
    {"id": "ug65p90000004n2z-att", "filename_prefix": "2010h22h_"},
    {"id": "gmcbt8000000f3yi-att", "filename_prefix": "2009h21a_"},
    {"id": "ug65p90000009bhl-att", "filename_prefix": "2009h21h_"},
]

# ファイルの種類とファイル名のパターン
# 午前/午後、問題/解答/採点講評などの組み合わせを定義
file_types = {
    "am": ["am_qs", "am_ans"],  # 午前問題、午前解答
    "pm": ["pm_qs", "pm_ans", "pm_cmnt"],  # 午後問題、午後解答、午後採点講評
    "am1": ["am1_qs", "am1_ans"],  # 午前I問題、午前I解答
    "am2": ["am2_qs", "am2_ans"],  # 午前II問題、午前II解答
    "pm1": ["pm1_qs", "pm1_ans", "pm1_cmnt"],  # 午後I問題、午後I解答、午後I採点講評
    "pm2": ["pm2_qs", "pm2_ans", "pm2_cmnt"],  # 午後II問題、午後II解答、午後II採点講評
}

# ファイル名のパターン
# ファイルの種類に対応する日本語名を取得するために使用
filename_patterns = {
    "qs": "問題",  # 問題
    "ans": "解答",  # 解答
    "cmnt": "採点講評",  # 採点講評
}


# 過去問をダウンロードする関数
def download_past_questions(exam_code):
    exam_name = exam_codes[exam_code]  # 試験区分名を取得

    for data in exam_data:  # 各試験年度/季節のデータをループ処理
        year = data["filename_prefix"][:4]  # 年を取得 (例: 2024)
        season = data["filename_prefix"][7]  # 季節を取得 (例: h, a, o)
        # 季節名を決定
        season_name = {"h": "春季", "a": "秋季", "o": "特別"}.get(season, "特別")

        # 試験区分別のディレクトリを作成
        create_dir = f"./DL/{exam_name}/{year}/{season_name}"
        if not path.exists(create_dir):
            makedirs(create_dir)

        for time_part in ["am", "pm"]:
            found_am_or_pm = False

            # 午前I/II、午後I/IIのファイルがあるか確認
            for i in range(1, 3):
                new_time_part = f"{time_part}{i}"
                # 新しいURLとファイル名を生成 (問題)
                new_url_qs = f"{base_url}{data['id']}/{data['filename_prefix']}{am_code}_{new_time_part}_qs.pdf"
                new_filename_qs = f"{year}{season_name}-{new_time_part.upper()}_{filename_patterns['qs']}.pdf"

                # 新しいURLとファイル名を生成 (解答)
                new_url_ans = f"{base_url}{data['id']}/{data['filename_prefix']}{am_code}_{new_time_part}_ans.pdf"
                new_filename_ans = f"{year}{season_name}-{new_time_part.upper()}_{filename_patterns['ans']}.pdf"

                # 問題と解答をダウンロード
                if download_file(new_url_qs, f"{create_dir}/{new_filename_qs}"):
                    found_am_or_pm = True
                    download_file(new_url_ans, f"{create_dir}/{new_filename_ans}")

                    # 採点講評のURLとファイル名を生成 (午後I/IIのみ)
                    if time_part == "pm":
                        new_url_cmnt = f"{base_url}{data['id']}/{data['filename_prefix']}{am_code}_{new_time_part}_cmnt.pdf"
                        new_filename_cmnt = f"{year}{season_name}-{new_time_part.upper()}_{filename_patterns['cmnt']}.pdf"
                        download_file(new_url_cmnt, f"{create_dir}/{new_filename_cmnt}")
                    break  # 問題が見つかったらループを抜ける

            # 午前 or 午後が見つからなかった場合、エラーを表示して停止
            if not found_am_or_pm:
                # am or pm のファイルがないか確認
                for file_type_suffix in file_types[time_part]:
                    # ... (URLとファイル名の生成処理は変更なし) ...

                    if download_file(url, f"{create_dir}/{filename}"):
                        found_am_or_pm = True
                        break  # ファイルが見つかったらループを抜ける

            if not found_am_or_pm:
                print(f"エラー: {year}{season_name} の {time_part} 問題が見つかりませんでした。")
                exit()

    print(f"{exam_name}のダウンロードが完了しました。")


# ファイルをダウンロードする関数
def download_file(url, filename):
    response = requests.head(url)  # HEADリクエストを送信してファイルの存在を確認
    if response.status_code == 200:  # ファイルが存在する場合 (ステータスコード200)
        urlretrieve(url, filename)  # ファイルをダウンロード
        print(f"{filename} を保存しました。")
        sleep(3)  # サーバーへの負荷を軽減するために3秒待機
        return True  # ダウンロード成功
    else:
        print(f"{filename} は見つかりませんでした。")
        return False  # ダウンロード失敗


# プログラムのエントリポイント
if __name__ == "__main__":
    while True:
        # ユーザーに試験区分コードの入力を促す
        inp = input(
            "ダウンロードしたい試験区分のコードを小文字で入力してください（例：ap）：\n修了する場合はqを入力してください。"
        )
        if inp in exam_codes:  # 入力されたコードが有効な場合
            download_past_questions(inp)  # 過去問をダウンロード
            break  # ループを抜ける
        elif inp == "q":  # 入力が "q" の場合
            print("プログラムを終了します。")
            exit()  # プログラムを終了
        else:  # 無効なコードが入力された場合
            print("試験区分のコードを入力してください。")  # エラーメッセージを表示

NameError: name 'am_code' is not defined